🛠 How It Works
Checks if a dataset already exists (radiation_Lankien.csv).
Finds missing dates by comparing available dates to expected ones.
Only fetches missing data instead of downloading everything again.
Appends new data to the existing dataset and saves it.
Now, you won't have to re-download everything, making the script efficient and faster. 🚀

In [8]:
import os
import requests
import pandas as pd
from datetime import datetime, timedelta

# Function to get NASA POWER solar radiation data
def get_solar_radiation(lat, lon, start_date, end_date):
    """
    Fetches solar radiation data from NASA POWER API.

    :param lat: Latitude of the location
    :param lon: Longitude of the location
    :param start_date: Start date (YYYYMMDD)
    :param end_date: End date (YYYYMMDD)
    :return: Pandas DataFrame with solar radiation data
    """
    url = f"https://power.larc.nasa.gov/api/temporal/daily/point?parameters=ALLSKY_SFC_SW_DWN&community=RE&longitude={lon}&latitude={lat}&start={start_date}&end={end_date}&format=JSON"

    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        
        if "ALLSKY_SFC_SW_DWN" in data['properties']['parameter']:
            records = data['properties']['parameter']['ALLSKY_SFC_SW_DWN']
            
            # Convert to DataFrame
            solar_df = pd.DataFrame(records.items(), columns=["date", "solar_radiation"])
            
            # Convert date column to datetime format
            solar_df["date"] = pd.to_datetime(solar_df["date"], format='%Y%m%d')
            
            return solar_df
        else:
            print("⚠️ No solar radiation data found in response.")
            return None
    else:
        print(f"❌ Error fetching data: {response.status_code}")
        print(response.text)
        return None

# 📍 Lankien (South Sudan)
latitude = 8.4281
longitude = 33.7833
start = "20150101"  # Fixed start date
end = (datetime.today() - timedelta(days=7)).strftime("%Y%m%d")  # 📅 Dynamic end date (7 days ago)

# 📂 Check if existing data is available
output_folder = "Solar_radiation_Dataset"
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, "radiation_Lankien.csv")
